In [2]:
import argparse

import numpy as np
import torch
import wandb
from deep_learning_rl_sm.trainer.trainer import Trainer
from deep_learning_rl_sm.neuralnets.minGRU_Reinformer import minGRU_Reinformer
from deep_learning_rl_sm.neuralnets.lamb import Lamb
from deep_learning_rl_sm.environments import connect_four

# TODO generate our datasets separately so we only have to load them here! input format!
env = connect_four.ConnectFour()
# maybe push generate sequences into the trainer class somewhere
data = torch.load("./data/offline_data.pt")

import sys
sys.argv = ['']

parser = argparse.ArgumentParser()
parser.add_argument("--model_type", choices=["reinformer"], default="reinformer")
parser.add_argument("--env", type=str, default=env)
parser.add_argument("--dataset", type=str, default="medium")
parser.add_argument("--num_eval_ep", type=int, default=10)
parser.add_argument("--max_eval_ep_len", type=int, default=1000)
parser.add_argument("--dataset_dir", type=str, default="data/d4rl_dataset/")
parser.add_argument("--context_len", type=int, default=5)
parser.add_argument("--n_blocks", type=int, default=4)
parser.add_argument("--embed_dim", type=int, default=256)
parser.add_argument("--n_heads", type=int, default=8)
parser.add_argument("--dropout_p", type=float, default=0.1)
parser.add_argument("--grad_norm", type=float, default=0.25)
parser.add_argument("--tau", type=float, default=0.99)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--lr", type=float, default=1e-4)
parser.add_argument("--wd", type=float, default=1e-4)
parser.add_argument("--warmup_steps", type=int, default=5000)
parser.add_argument("--max_train_iters", type=int, default=10)
parser.add_argument("--num_updates_per_iter", type=int, default=5000)
parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--seed", type=int, default=2024)
parser.add_argument("--init_temperature", type=float, default=0.1)
parser.add_argument("--eps", type=float, default=1e-8)
parser.add_argument("--use_wandb", action='store_true', default=False)
args = parser.parse_args()

if args.use_wandb:
    wandb.login()
    wandb.init(
        name=args.env + "-" + args.dataset,
        project="Reinformer",
        force=True,
        config=vars(args)
    )
# TODO have discrete set based on environment choice instead of manual
discrete = True
# TODO explore different target entropies
# entropy to encourage exploration in RL typically -action_dim for continuous actions and -log(action_dim) when discrete
target_entropy = -np.log(np.prod(env.action_dim)) if discrete else -np.prod(env.action_dim)
args = vars(args)
model = minGRU_Reinformer(state_dim=env.state_dim, act_dim=env.action_dim, n_blocks=args["n_blocks"],
                          h_dim=args["embed_dim"], context_len=args["context_len"], n_heads=args["n_heads"],
                          drop_p=args["dropout_p"], init_tmp=args["init_temperature"],
                          target_entropy=target_entropy)
optimizer = Lamb(
            model.parameters(),
            lr=args["lr"],
            weight_decay=args["wd"],
            eps=args["eps"],
        )
scheduler = torch.optim.lr_scheduler.LambdaLR(
            optimizer,
            lambda steps: min((steps + 1) / args["warmup_steps"], 1)
        )
# expect the next line to cause problems (need padding to make this work)



/var/folders/ml/07xg0jzx2t3c2d2nq1bk1njc0000gn/T/ipykernel_36145/3545371732.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("./data/offline_data.pt")


In [3]:
for key in data:
    print(key)

states
actions
rewards
dones
time_steps
action_masks
returns_to_go


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data):
        # Storing each data item separately for easy access
        self.states = data['states']
        self.actions = data['actions']
        self.rewards = data['rewards']
        self.dones = data['dones']
        self.time_steps = data['time_steps']
        self.action_masks = data['action_masks']
        self.returns_to_go = data['returns_to_go']
        
    def __len__(self):
        # Return the number of samples (assuming all lists have the same length)
        return len(self.states)

    def __getitem__(self, idx):
        # Return a tuple of each item type for a given index
        return (self.states[idx,:,:], 
                self.actions[idx,:,:], 
                self.rewards[idx,:,:], 
                self.dones[idx,:,:], 
                self.time_steps[idx,:,:], 
                self.action_masks[idx,:,:], 
                self.returns_to_go[idx,:,:])


In [5]:
dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

# Getting a sample batch

    
    

In [99]:
iter_data=iter(dataset)
next(iter_data)[1].shape

torch.Size([21, 1])

In [7]:
iter_data=iter(dataloader)


In [21]:
item = next(iter_data)
(states,
actions,
rewards,
dones,
time_steps,
action_masks,
returns_to_go) = item

In [ ]:
trainer = Trainer(model=model, dataset=data, optimizer=optimizer, scheduler=scheduler, parsed_args=args)
trainer.train(args)

In [3]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Each item in the batch is a sequence
    padded_sequences = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_sequences

In [51]:
from torch.utils.data import DataLoader

# Assuming `sequences` is the list of sequences generated by your environment
dataset = SequenceDataset(sequences)

In [42]:
trainer = Trainer(model=model, dataset=dataset, optimizer=optimizer, scheduler=scheduler, parsed_args=args)
trainer.train(args)

StopIteration: 

In [52]:
data_loader = DataLoader(
            dataset=dataset,
            batch_size=8,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
        )
iterate_data = iter(data_loader)

In [53]:
next(iterate_data)

TypeError: len() of unsized object

In [57]:
sequences[0][1]

[array(0),
 array([1, 1, 1, 1, 1, 1, 1], dtype=int8),
 array(1),
 array(0),
 np.int64(1),
 array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 2., 0.]])]

In [60]:
seq = env.generate_seq(no_sequences=1)

In [61]:
seq

[[array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]),
  [array(0),
   array([1, 1, 1, 1, 1, 1, 1], dtype=int8),
   array(6),
   array(0),
   np.int64(-1),
   array([[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 2.],
          [0., 0., 0., 0., 0., 0., 1.]])],
  [array(1),
   array([1, 1, 1, 1, 1, 1, 1], dtype=int8),
   array(3),
   array(0),
   np.int64(-1),
   array([[0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 2.],
          [0., 0., 2., 1., 0., 0., 1.]])],
  [array(2),
   array([1, 1, 1, 1, 1, 1, 1], dtype=int8),
   array(1),
   ar